# Generate images for the restaurant   

After the dishes, we also generate images for the restaurant appearance.

## optional: install more libs

In [1]:
print('hello')

hello


In [1]:
# install diffusers from source if needed
# as the fluxpipeline is brand new

# !pip install git+https://github.com/huggingface/diffusers

In [2]:
# !pip uninstall transformer-engine -y

In [3]:
# !pip install transformers

In [4]:
# !pip install sentencepiece

In [5]:
# from huggingface_hub import notebook_login
# notebook_login()

In [6]:
# !pip uninstall apex -y

In [8]:
# !pip install optimum-quanto

In [9]:
# !pip install -U torchvision

In [10]:
# !pip3 install -U xformers --index-url https://download.pytorch.org/whl/cu121

### setup pipeline

In [5]:
import torch
from diffusers import FluxTransformer2DModel, FluxPipeline
from transformers import T5EncoderModel, CLIPTextModel
from optimum.quanto import freeze, qfloat8, quantize

bfl_repo = "black-forest-labs/FLUX.1-dev"
dtype = torch.bfloat16

transformer = FluxTransformer2DModel.from_single_file("https://huggingface.co/Kijai/flux-fp8/blob/main/flux1-dev-fp8.safetensors", torch_dtype=dtype)
quantize(transformer, weights=qfloat8)
freeze(transformer)

text_encoder_2 = T5EncoderModel.from_pretrained(bfl_repo, subfolder="text_encoder_2", torch_dtype=dtype)
quantize(text_encoder_2, weights=qfloat8)
freeze(text_encoder_2)

pipe = FluxPipeline.from_pretrained(bfl_repo, transformer=None, text_encoder_2=None, torch_dtype=dtype)
pipe.transformer = transformer
pipe.text_encoder_2 = text_encoder_2

pipe.enable_model_cpu_offload()

# prompt = "A cat holding a sign that says hello world"
# image = pipe(
#     prompt,
#     guidance_scale=1.5,
#     height=1024,
#     width=1024,
#     output_type="pil",
#     num_inference_steps=20,
#     generator=torch.Generator("cpu").manual_seed(0)
# ).images[0]

# image.save("flux-fp8-dev.png")

/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


### test pipeline

In [12]:
# prompt = "A cat holding a sign that says 'i am stupid'"
# image = pipe(
#     prompt,
#     guidance_scale=1.5,
#     height=1024,
#     width=1024,
#     output_type="pil",
#     num_inference_steps=20,
#     generator=torch.Generator("cpu").manual_seed(0)
# ).images[0]

# image.save("flux-fp8-dev-2.png")

In [13]:
# prompt = "A cat holding a sign that says 'i love cheese'"
# image = pipe(
#     prompt,
#     guidance_scale=1.5,
#     height=1024,
#     width=1024,
#     output_type="pil",
#     num_inference_steps=20,
#     generator=torch.Generator("cpu").manual_seed(0)
# ).images[0]

# image.save("flux-fp8-dev-3.png")

In [14]:
# prompt = "picture of a delicious cheeseburger with fries on the side, in the forest, nature background"
# image = pipe(
#     prompt,
#     guidance_scale=1.5,
#     height=1024,
#     width=1024,
#     output_type="pil",
#     num_inference_steps=20,
#     generator=torch.Generator("cpu").manual_seed(0)
# ).images[0]

# image.save("flux-fp8-dev-4.png")

In [15]:
# prompt = "Amuse-Bouche Crispy Parmesan waffle with tomato and basil gelée, 4k food photography, top down picture of a plate, michelin starred dish"
# image = pipe(
#     prompt,
#     guidance_scale=1.5,
#     height=1024,
#     width=1024,
#     output_type="pil",
#     num_inference_steps=25,
#     generator=torch.Generator("cpu").manual_seed(0)
# ).images[0]

# image.save("food-test-2.png")

In [5]:
# prompt = "L'Etoile d'Or offers a luxurious and intimate dining experience, adorned with opulent gold accents, crystal chandeliers, and elegant silk drapery. The soft lighting and plush velvet seating create an atmosphere of romance and sophistication, while the floor-to-ceiling windows offer breathtaking views of the Eiffel Tower."
# image = pipe(
#     prompt,
#     guidance_scale=2,
#     height=1024,
#     width=1024,
#     output_type="pil",
#     num_inference_steps=25,
#     generator=torch.Generator("cpu").manual_seed(0)
# ).images[0]

# image.save("restaurant-test-6.png")

We find that 25 steps with the above prompt template is the ideal config to run all 288 food prompts.  

### Process all 41 prompts

We load a datasets of prompts and filenames and pass it to the image generation function

In [2]:
# we set the seed to a fixed 42
def from_prompt_to_file(prompt, filename):
    image = pipe(
        prompt,
        guidance_scale=1.5,
        height=1024,
        width=1024,
        output_type="pil",
        num_inference_steps=25,
        generator=torch.Generator("cpu").manual_seed(42)
    ).images[0]

    image.save('images/'+filename)

In [7]:
# load the prompt dataset
import pandas as pd

prompt_df = pd.read_csv('restaurant_prompts_and_filenames.csv', index_col=0)
prompt_df.head(5)

,prompt,filename
0,L'Etoile d'Or offers a luxurious and intimate ...,L'Etoile_d'Or.jpg
1,Sakura No Hana provides an exquisite and seren...,Sakura_No_Hana.jpg
2,"Taman Sari, meaning 'Water Garden' in Indonesi...",Taman_Sari.jpg
3,Le Lagon offers an unparalleled tropical dinin...,Le_Lagon.jpg
4,"Inti Raymi, named after the Inca festival cele...",Inti_Raymi.jpg


In [8]:
from tqdm.auto import tqdm

tqdm.pandas()

prompt_df.progress_apply(lambda row: from_prompt_to_file(row['prompt'], row['filename']), axis=1)

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (83 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['views of mount otemanu.']


...

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['each course is paired with a carefully selected wine, enhancing the flavors and providing a truly unforgettable dining experience.']


  0%|          | 0/25 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['sophisticated menu, created by chef pierre gagnaire, showcases classic french cuisine with a modern twist, using the finest ingredients like caviar, truffles, and foie gras. the impeccable, attentive service and extensive wine list, including rare vintages and exclusive labels, complete the unforgettable dining experience at the imperial room.']


  0%|          | 0/25 [00:00<?, ?it/s]

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
dtype: object